<a href="https://colab.research.google.com/github/folkn/Numerical-Methods/blob/master/Linear_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLVING LINEAR SYSTEMS in PYTHON
**Warapong Narongrit and Techin Choklertwatana**

references
https://numericalmethodsece101.weebly.com/doolittlersquos-method.html
https://numericmethod.weebly.com/uploads/2/5/0/8/25086597/lkm-8-e.pdf

We start by importing the necessary libraries. Namely **numpy** and **scipy** for linear algebra functions

In [0]:
import numpy as np
from scipy import linalg
np.set_printoptions(floatmode='maxprec_equal')
import pprint

We are interested in solving simultaneous linear equations.
FIRST, we input the simultaneous equation in matrix form $Ax=b$ :

$$
\begin{bmatrix}
    A_{11}       & A_{12} & A_{13} & \dots & A_{1n} \\
    A_{21}       & A_{22} & A_{23} & \dots & A_{2n} \\
    A_{d1}       & A_{d2} & A_{d3} & \dots & A_{dn}
\end{bmatrix} 
\begin{bmatrix}
     x_1 \\
     \dots \\
     x_n 
\end{bmatrix}
= \begin{bmatrix}
    b_1 \\
    \dots \\
    b_n
\end{bmatrix}
$$

Input the matrix to be solved here. $A$ as a square matrix, $B$ as a column vector


In [165]:
#INPUT MATRIX TO BE SOLVED HERE#
A = np.array([[1.,-0.25, -0.25, 0],[-0.25,1,0,-0.25],[-0.25,0,1,-0.25],[0,-0.25,-0.25,1]])
b = np.array([[50.], [50.], [25.], [25.]])




print('Input:\nA=\n' + str(A)); print('b=\n' + str(b))   #Prints matrices
print('\nOutput:\neigenvalues= ' + str(np.linalg.eigvals(A))) #Checks eigenvalues
print ('\nActual Solution: X=\n' + str(np.linalg.solve(A,b))) #Exact solution as calculated by numpy

Input:
A=
[[ 1.00 -0.25 -0.25  0.00]
 [-0.25  1.00  0.00 -0.25]
 [-0.25  0.00  1.00 -0.25]
 [ 0.00 -0.25 -0.25  1.00]]
B=
[[50.]
 [50.]
 [25.]
 [25.]]

Output:
eigenvalues= [0.5 1.0 1.5 1.0]

Actual Solution: X=
[[87.5]
 [87.5]
 [62.5]
 [62.5]]


## Analytical Solution using Gauss Elimination

## Analytical Solution using LU Factorization 

This method uses the idea that all matrices (except those with some diagonal elements being zero) can be factorized into products Lower and upper triangular matrices:

$$A = LU$$ 

$$
\begin{bmatrix}
    A_{11}       & A_{12} & A_{13}  \\
    A_{21}       & A_{22} & A_{23}  \\
    A_{31}       & A_{32} & A_{33} 
\end{bmatrix} =
\begin{bmatrix}
    l_{11}       & 0 & 0  \\
    l_{21}       & l_{22} & 0  \\
    l_{31}       & l_{32} & l_{33} 
\end{bmatrix} 
\begin{bmatrix}
    u_{11}       & u_{12} & u_{13}  \\
    0       & u_{22} & u_{23}  \\
    0       & 0 & 0
\end{bmatrix}
$$

There is more than one way of factorizing, so three **LU Factorization** methods are used:
1. **Doolittle** - all Diagonals of $L$ are $1$    ($l_{11} = l_{22} = l_{nn} = 1$)
2. **Crout** - all Diagonals of $U$ are $1$     ($u_{11} = u_{22} = u_{nn} = 1$)
3. **Cholesky** - $U = L^\text{T}$ or equivalently $L=U^\text{T}$ such that $A=LL^\text{T}$ (*works only if A=positive definite(**positive eigenvalues**) and symmetric **($A=A^\text{T}$)**
---
**The procedure to solving linear systems is:**
1. LU-Factorize the coefficients matrix $A$ using one of the methods
2. Let $LY = b$ where $Y$ is an intermediate variable matrix. Solve for $Y$
3. Let $UX = Y$ then solve for the unknowns $X$ 

(This works because $AX=b, LUX=b, \text{let }Y=UX, \text{so }LY=b$)

I first create functions for steps 2 and 3 since all methods will require these steps

In [0]:
def LY_B (L, B) : #Step 2
    Y =  np.matmul((np.linalg.inv(L)), B)
    return Y

def UX_Y (U, Y) : #Step 3
    X = np.matmul(np.linalg.inv(U) , Y)
    return X

Now I will perform the three methods of factorization methods

### 1. Doolittle's Method https://www.codewithc.com/lu-decomposition-algorithm-flowchart/ https://vismor.com/documents/network_analysis/matrix_algorithms/S4.SS2.php ###

In the code, we convert the $LU$ vector multiplication into a dot multiplcation problem for better scaling.

In [0]:
#1. Doolittle Factorization https://stackoverflow.com/questions/48715594/why-is-my-profs-version-of-lu-decomposition-faster-than-mine-python-numpy
def doolittle(A) : 
    N = np.size(A, 0)    # Number of Rows (Equations)
    
    # Create empty L U matrices with the dimensions of A
    L = np.zeros_like(A)  
    U = np.zeros_like(A)


    for k in range(N):   #For each equation (row) k
        #Determining Diagonal Elements
        L[k, k] = 1      #Set the L diagonal to 1  (Doolittle Requirement)
        U[k, k] = (A[k, k] - np.dot(L[k, :k], U[:k, k])) / L[k, k] 
        #Determining Other Elements
        for j in range(k+1, N):
            U[k, j] = (A[k, j] - np.dot(L[k, :k], U[:k, j])) / L[k, k]
        for i in range(k+1, N):
            L[i, k] = (A[i, k] - np.dot(L[i, :k], U[:k, k])) / U[k, k]
    return L, U
    
L, U = doolittle(A);
print ('L=\n' +str(L)); print('U=\n' + str(U))

#Step 2, 3, ANS
Y= LY_B(L,B)
X = UX_Y(U,Y)
print ('LY=B\nY=\n' + str(Y)); print ('UX=Y\nX=\n' +str(X) +'\tANS')


L=
[[1.  0. ]
 [2.5 1. ]]
U=
[[2.  1. ]
 [0.  4.5]]
LY=B
Y=
[[ 11. ]
 [-14.5]]
UX=Y
X=
[[ 7.11111111]
 [-3.22222222]]	ANS


In [0]:
#Doolittle using SciPy's functinos
def scipy_plu(A) :
    L, U = linalg.lu(A, True)
    return L,U
L,U = scipy_plu(A)
print ('L=\n' +str(L)); print('U=\n' + str(U))

 #Step 2, 3, ANS
Y= LY_B(L,B)
X = UX_Y(U,Y)
print ('LY=B\nY=\n' + str(Y)); print ('UX=Y\nX=\n' +str(X) +'\tANS')

L=
[[0.4 1. ]
 [1.  0. ]]
U=
[[ 5.   7. ]
 [ 0.  -1.8]]


NameError: ignored

### 2. Crout's Method

In [0]:
#2. Crout Factorization 


### 3. Cholesky's Method

In [0]:
#3. Cholesky's Method
def cholesky(A) :
    L = np.linalg.cholesky(A) #wow!
    U = L.T #Cholesky requirement U=transpose(L)
    return L,U

#Try the cholesky, and catch any errors
try:
    L,U = cholesky(A)
except Exception as err:
    print (err)
print ('L=\n' +str(L)); print('U=\n' + str(U))
#Step 2, 3, ANS
Y= LY_B(L,B)
X = UX_Y(U,Y)
print ('LY=B\nY=\n' + str(Y)); print ('UX=Y\nX=\n' +str(X) +'\tANS')

L=
[[2.23606798 0.        ]
 [1.34164079 0.4472136 ]]
U=
[[2.23606798 1.34164079]
 [0.         0.4472136 ]]
LY=B
Y=
[[0.4472136 ]
 [3.13049517]]
UX=Y
X=
[[-4.]
 [ 7.]]	ANS


## Numerical Solutions using Iterations

This method uses repeated calculations to **approximate** the solutions. It is normally used for matrices with large diagonal entries and those with many zero coefficients.

### 1. Jacobi Iteration
Assumes that the diagnoal elements are non-zero ($a_{nn}\ne 0$)https://www3.nd.edu/~zxu2/acms40390F12/Lec-7.3.pdf https://www.kth.se/social/files/5885d039f2765429974418ce/Lab1.pdf
https://gist.github.com/angellicacardozo/3a0891adfa38e2c4187612e57bf271d1


For a system such as
$$x + y + z = 1 \\2x+4y+8z=2 \\3x+12y+20z=3\\$$
We try to isolate each variable in each equation as such:
$$x=1-y-z(1)\\y=\frac{2-2x-8z}{4}(2)\\z=\frac{3-3x-12y}{20}(3)
$$

Now we come up with an *initial guess* of the solutions, $x^{(0)}$

In our example, let's suppose $[x^{(0)}, y^{(0)}, z^{(0)}]=[0,0,0]$

We substitute the initial guess into $(1),(2),(3)$, which will give us $x^{(1)}$ (our first iteration)

We repeat this step until we are satisfied of our solutions.

**In matrix form**, this is shown as

$$A=\begin{bmatrix}
    1       & 1 & 1  \\
    2       & 4 & 8  \\
    3       & 12 & 20 
\end{bmatrix}
, B=
\begin{bmatrix}
    1      \\
    2      \\
    3     
\end{bmatrix}
$$


Then we attempt to create equations (1) (2) (3)  by separating the diagonals

$$
D=\begin{bmatrix}
    1      \\
    4      \\
    12     
\end{bmatrix} \text{(Diagonal elements)}
$$ 

$$
R=\begin{bmatrix}
    1       & 1 & 1  \\
    2       & 4 & 8  \\
    3       & 12 & 20 
\end{bmatrix} -
\begin{bmatrix}
    1       & 0& 0  \\
    0       & 4 & 0 \\
    0       & 0 & 20 
\end{bmatrix}
=\begin{bmatrix}
    0       & 1 & 1  \\
    2       & 0 & 8  \\
    3       & 12 & 0 
\end{bmatrix}
$$

$$
x^{(n)} = \frac{(RX - B)}{D} \text{  (elementwise division)}
$$
In code, the matrix multiplication RX is performed row-wise as a dot product problem to simplify code.

The initial guess and iterations are placed in matrix $x$ and substituted as above.


If the diagonals of $A$ are all $1$, the iteration can be written as $$x^{(n+1)}=B+(I-A)x^{(m)}$$. Otherwise, it is written as 
$$x_i^{(n)}=\frac{1}{a_{ii}}[\sum^{eq}_{j=1, j\ne i}(-a_{ij}x^{(n)})+B]$$



In [209]:
###Please Input 'Guess' Here as a column vector###
X = np.array([[100],[100],[100],[100]])

Iterations = 100            # Stop calculation if either tolerance or iteration condition is met
Tolerance = 0.0001 #Absolute Tolerance          Set to <=0 to ignore
Print_Values = True #Print value of each step
######################################################################
print('Guess X=\n'+str(X) + '\nfor A=\n' +str(A) + '  using ' +str(Iterations) + ' Iterations, Abs.Tol=' + str(Tolerance))

def jacobi(A, b, N, x, tol):   
    eq = np.size(A, 0)    # Number of Rows (Equations)
    D = diag(A) #Obtain the diagonals as a row vector
    R = A - diagflat(D) #Obtain matrix A without its diagonals
                                                                                                                                                                    
    for i in range(N): #Iterations
        x_old = x #store previous iteration
        x_max=0 #For error condition
        x = (b - dot(R,x)) / D #new iteration
        print(x)
       
        if np.amax(abs(x-x_old)) > x_max:
          x_max = np.amax(abs(x-x_old))
        if x_max < tol:
          print('Tolerance condition met before number of iterations', i, x_max)
          return x, i, x_max
      
          
        if Print_Values is True: #Print values of each Iteration
          if i is 0:
            print('\n\n(Iteration, Iteration Values, Max Error)')  
          print (i+1, str(x), x_max)
    print('Number of iterations condition met before tolerance condition', i, x_max)
    return x, i, x_max
    

b = B


sol,iterN,x_max = jacobi(A,b.flatten(), Iterations, X.flatten() , Tolerance)
print('\n\nX='+ str(sol) + '\nwith a final tolerance of ' + str(x_max) + ' after ' + str(iterN) + ' iterations')

Guess X=
[[100]
 [100]
 [100]
 [100]]
for A=
[[ 1.00 -0.25 -0.25  0.00]
 [-0.25  1.00  0.00 -0.25]
 [-0.25  0.00  1.00 -0.25]
 [ 0.00 -0.25 -0.25  1.00]]  using 100 Iterations, Abs.Tol=0.0001
[100. 100.  75.  75.]


(Iteration, Iteration Values, Max Error)
(1, '[100. 100.  75.  75.]', 25.0)
[93.75 93.75 68.75 68.75]
(2, '[93.75 93.75 68.75 68.75]', 6.25)
[90.625 90.625 65.625 65.625]
(3, '[90.625 90.625 65.625 65.625]', 3.125)
[89.0625 89.0625 64.0625 64.0625]
(4, '[89.0625 89.0625 64.0625 64.0625]', 1.5625)
[88.28125 88.28125 63.28125 63.28125]
(5, '[88.28125 88.28125 63.28125 63.28125]', 0.78125)
[87.890625 87.890625 62.890625 62.890625]
(6, '[87.890625 87.890625 62.890625 62.890625]', 0.390625)
[87.6953125 87.6953125 62.6953125 62.6953125]
(7, '[87.6953125 87.6953125 62.6953125 62.6953125]', 0.1953125)
[87.59765625 87.59765625 62.59765625 62.59765625]
(8, '[87.59765625 87.59765625 62.59765625 62.59765625]', 0.09765625)
[87.548828125 87.548828125 62.548828125 62.548828125]
(9, '[87.5

### 2. Gauss-Seidel Iteration
https://austingwalters.com/gauss-seidel-method/
http://milesbarnhart.com/portfolio/python/python-gauss-seidel-approximation-method/

With the Jacobi method, we use the same iteration values $x^{(n)}$ until we are done with the iteration.

With the Gauss-Seidel method, we use the newly calculated value $x^{(n+1)}_i$ right after it is known. For example, once we determined $x_1^{(n+1)}$ from the first equation, we use it to find $x_2^{(n+1)}$ in the second equation right away. 

This is similar to the previous method, but requires certain modifications to the equation.

In matrix form, we wil have to separate the lower and upper triangles of matrix $A$ to account for this change. The matrix equation becomes $$x^{(n+1)} = B - L x^{(n+1)} - Ux^{(n)}$$

In [0]:
def gauss(A, b, x, n):

    L = np.tril(A) #Lower triangle
    U = A - L
    print (L)
    print(U)
    print (np.matmul(U,x))
    for i in range(n):
        x = np.matmul(np.linalg.inv(L), b - np.matmul(U, x))
        #print str(i),
       # print(x)
    return x

#A = np.array([[4.0, -2.0, 1.0], [1.0, -3.0, 2.0], [-1.0, 2.0, 6.0]])
#b = [1.0, 2.0, 3.0]
#x = [1, 1, 1]

n = 100

print gauss(A, b.flatten(), X.flatten(), n)

In [207]:
def gauss(A, b, x, n):

    L = np.tril(A) #Lower triangle
    U = A - L
    print (L)
    print(U)
    print (np.matmul(U,x))
    for i in range(n):
        x = np.matmul(np.linalg.inv(L), b - np.matmul(U, x))
        #print str(i),
       # print(x)
    return x

#A = np.array([[4.0, -2.0, 1.0], [1.0, -3.0, 2.0], [-1.0, 2.0, 6.0]])
#b = [1.0, 2.0, 3.0]
#x = [1, 1, 1]

n = 100

print gauss(A, b.flatten(), X.flatten(), n)

[[ 1.00  0.00  0.00  0.00]
 [-0.25  1.00  0.00  0.00]
 [-0.25  0.00  1.00  0.00]
 [ 0.00 -0.25 -0.25  1.00]]
[[ 0.00 -0.25 -0.25  0.00]
 [ 0.00  0.00  0.00 -0.25]
 [ 0.00  0.00  0.00 -0.25]
 [ 0.00  0.00  0.00  0.00]]
[-50. -25. -25.   0.]
[87.5 87.5 62.5 62.5]
